# Практика
Используемые библиотеки

In [145]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import requests # for web-download
import io # for web-download
import re # for data processing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [7]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')


In [99]:
# Ваше решение
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [131]:
# Функция для выделения числовых границ из столбца "Норматив"
def split_norm(str):
    str = str.replace(',', '.') 
    string = re.split(r'\s|-', f'{str}')
    if len(string) == 3:
        return 0.0, float(string[-1])
    if len(string) == 4:
        return float(string[-2]), float(string[-1])
    
# Функция для сравнения значения из столбца "Результата анализа" с границами из столбца "Норматив"    
def define_norm(value, data):
    segment = split_norm(data)
    if re.search(r'[А-Яа-яЁё]\s*', value):
        return 'нельзя определить'
    else:
        if float(value)>= segment[0] and float(value) <= segment[1]:
            return 'в норме'
        else:
            return 'не в норме'

In [135]:
df['Вывод'] = df.apply(lambda x: define_norm(value = x['Результат анализа'], data = x['Норматив']), axis=1)
df.set_index('Показатель')

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,в норме
Запах,баллы,1,не более 2-3,не в норме
Цветность,градусы,б/цвета,не более 30,нельзя определить
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",в норме
Нитраты (по NO3),мг/дм3,24,не более 45,в норме
Фосфаты (P),мг/дм3,0.36,"не более 3,5",в норме
Хлориды (Cl),мг/дм3,200,не более 350,в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [155]:
# Ваше решение

number_of_apples = 4
number_of_oranges = 5

A = {'All oranges': 0, 'Not all oranges': 0}
n = 10000000


for i in tqdm(range(n)):
    
    # Заполняем ящик 5 апельсинами и 4 яблоками
    Box = number_of_oranges*['orange'] + number_of_apples*['apple']
    
    # Перемешиваем фрукты в коробке
    random.shuffle(Box)
    combination = []
    
    # Начинаем вытаскиваем по фрукту из корзины
    for j in range(3):
        combination.append(random.choice(Box))
        
        # Удаляем из коробки фрукт, который только что вытащили из коробки                
        Box.remove(combination[j])
        
    # Если есть хотя бы одно яблоко в комбинации, то увеличиваем значение количества комбинаций "Not all oranges"    
    if 'apple' in combination:
        A.update({'Not all oranges': A['Not all oranges']+1})
        
    # Если в комбинации все апельсины, то увеличиваем значение количества комбинаций "Аll oranges"    
    else:
        A.update({'All oranges': A['All oranges']+1})
        
# Проверяем, что количество проведенных экспериментов равно количеству запланированных экспериментов        
assert n == A['All oranges']+A['Not all oranges']        
print(A)

# Считаем статистику выбора трех апельсинов подряд (количество раз выпадения трех апельсинов делим на общее количество экспериментов)
print('Вероятность, что все три фрукта – апельсины равна', A['All oranges']/n)

  0%|          | 0/10000000 [00:00<?, ?it/s]

{'All oranges': 1191721, 'Not all oranges': 8808279}
Вероятность, что все три фрукта – апельсины равна 0.1191721


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [154]:
# Ваше решение

number_non_standard = 3
number_standard = 10 - number_non_standard

A = {'Two details': 0, 'Another case': 0}
n = 10000000

for i in tqdm(range(n)):
    
    # Заполняем ящик стандартными и нестандартными деталями
    Box = number_non_standard*['non_standard'] + number_standard*['standard']
    random.shuffle(Box)
    combination = []
    for j in range(2):
        choice = random.choice(Box)
        combination.append(choice)
            
        # Удаляем из коробки деталь, которую только что вытащили из коробки                
        Box.remove(combination[j])
        
    # Если первая деталь нестандартная, а вторая стандартная, то увеличиваем значение количества комбинаций "Two details"    
    if combination[0] == 'non_standard' and combination[1] == 'standard':
        A.update({'Two details': A['Two details']+1})
        
    # Если последовательность деталь другая, то увеличиваем значение количества комбинаций "Another case"    
    else:
        A.update({'Another case': A['Another case']+1})
        
# Проверяем, что количество проведенных экспериментов равно количеству запланированных экспериментов        
assert n == A['Another case']+A['Two details']        
print(A)

# Считаем статистику, что мастер проверит ровно две детали (количество случаев, когда мастер проверит ровно 2 детали,
# делим на общее количество экспериментов)
print('Вероятность, что мастер проверит ровно две детали', A['Two details']/n)

  0%|          | 0/10000000 [00:00<?, ?it/s]

{'Two details': 2333800, 'Another case': 7666200}
Вероятность, что мастер проверит ровно две детали 0.23338
